In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import fiona
import pyproj
import contextily
import plotly.express as px
from shapely import wkt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
path = "/srv/data/my_shared_data_folder/cpr/"

dir_list = os.listdir(path)
#dir_in = os.listdir(os.getcwd())
#directory = pd.Series(dir_in)
#print(dir_in)
print(dir_list)

['california_counties_census_data.csv', 'cleaned_county_data.csv', 'R13391251_SL140.csv', 'R13391250_SL050.csv', 'california_tracts.gpkg', 'pur2020_nonag.parquet', 'cleaned_tract_data.csv', 'cleaned_elsd_data.csv', 'california_zctas.gpkg', 'california_counties.gpkg', 'vision_providers_geocoded.csv', 'california_watersheds.gpkg', 'california_zcta_census_data.csv', 'R13391252_SL950.csv', 'new_test_file.gpkg', 'readme.md', 'california_elementary_school_districts.gpkg', 'pur2020_ag.parquet', 'california_tracts_census_data.csv', 'cleaned_zcta_data.csv', 'california_elementary_schools_census_data.csv', 'R13391293_SL860.csv', 'another_copy_pur2020_ag.parquet']


# READ
This notebook consolidates all code used to create the files used for visualization and analysis. Any checkpoint is an optional place where you can choose to save a file, but is not a final file that ended up being used for analysis. This is a consolidation: the file are to be created one at a time, which will be indicated with each new file name, as well as the code to load in prior created files. The files documented in this notebook are as follows:
1. county_w_geo.geojson
2. tract_w_geo.geojson
3. tracts_w_geo+.geojson
4. demographic_merge.geojson
5. Pesticide_5.csv
6. pesticide_5_meridian.csv
7. Pesticide_5_w_tracts.csv
8. Pesticide_5_w_tracts+.csv
9. places_w_MTownshipRange

## We typically default to using the crs 4326 for compatibility with plotly choropleths
- - -

# 1. county_w_geo.geojson
uses files: 
1. california_counties.gpkg
2. cleaned_county_data.csv

In [2]:
# loading in county geometry
geometry = gpd.read_file(path + '/california_counties.gpkg', engine = 'fiona')

In [8]:
# rename for clarity
geometry.rename(columns = {'GEOID': 'StateCountyFIPS'},inplace = True)

In [3]:
# loading in county data
data = pd.read_csv(path + '/cleaned_county_data.csv')

In [4]:
# functions for cleaning "data"
# these help reformat the numeric data as strings, which is useful in the case of FIPS codes
# please look up FIPS codes to understand their makeup for this merge
def fix_StCounty(string):
    if (len(string) == 4):
        fix = '0' + string
    elif (len(string) == 5):
        fix = string
    else: return np.nan
    
    return fix

def fix_St(string):
    if (len(string) == 1):
        fix = '0' + string
    elif (len(string) == 2):
        fix = string
    else: return np.nan
    
    return fix

def fix_Cnty(string):
    if (len(string) == 1):
        fix = '00' + string
    elif (len(string) == 2):
        fix = '0' + string
    elif (len(string) == 3):
        fix = string
    else: return np.nan
    
    return fix

In [5]:
# reformatting as FP strings
# two daisy chained operations: turning numeric to string and applying string cleaning functions
data['FIPS'] = data['FIPS'].apply(lambda x: str(x)).apply(fix_StCounty)
data['state_fips'] = data['state_fips'].apply(lambda x: str(x)).apply(fix_St)
data['county'] = data['county'].apply(lambda x: str(x)).apply(fix_Cnty)

# fixing names to improve labeling
data.rename(columns = {'FIPS': 'StateCountyFIPS','state_fips': 'STATEFP','county': 'COUNTYFP'}, inplace = True)

In [6]:
# dropping empty tract column 
data.drop(columns = ['tract'], inplace = True)

In [9]:
merge = geometry.merge(data, on=['StateCountyFIPS','STATEFP','COUNTYFP'])

In [10]:
# adding geometry type for later filtering
merge['geometry_type'] = 'county'

In [12]:
# adding explicit county name for later filtering
merge['county_name'] = merge['NAME']

In [13]:
# changing crs for later plotting
merge.to_crs(pyproj.CRS.from_epsg(4326), inplace = True)

In [18]:
merge.head(2)

,STATEFP,COUNTYFP,COUNTYNS,StateCountyFIPS,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,state,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino,geometry_type,county_name
0,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,NaN,NaN,NaN,A,2468694587,23299110,+39.5769252,-120.5219926,"MULTIPOLYGON (((-120.65560 39.69357, -120.6555...",ca,05000US06091,Sierra County,953.168327,3079,2099,908,474,728,23,56152,2692,2,28,0,0,0,12,345,475,1151,0.432587,0.153946,0.031593,0.412685,0.874310,0.000650,0.009094,0.000000,0.000000,0.00000,0.003897,0.112049,county,Sierra
1,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,NaN,A,2499183617,76073827,+38.4500114,-121.3404409,"MULTIPOLYGON (((-121.18858 38.71431, -121.1873...",ca,05000US06067,Sacramento County,965.273130,1571767,583858,323018,84824,370480,35387,76422,674153,145916,4057,261202,17387,5925,88395,374732,371775,559036,0.553248,0.053967,0.095517,0.665029,0.428914,0.092836,0.002581,0.166184,0.011062,0.00377,0.056239,0.238414,county,Sacramento


In [19]:
merge.to_file('county_w_geo.geojson',driver = 'GeoJSON', crs = merge.crs)

In [23]:
# test = gpd.read_file('./Files/county_w_geo.geojson')
# test

- - -
- - -

# 2 - 3. tract_w_geo.geojson & tract_w_geo+.geojson
uses files:
1. tl_2021_06_tract.shp
2. cleaned_tract_data.csv

In [2]:
# loading in tract geometry
# note that you need all files in the shape folder for this to work properly
geometry = gpd.read_file("../../data/tract_shape/tl_2021_06_tract.shp")

In [3]:
# renaming for consistency
geometry.rename(columns = {'TRACTCE':'TRACTFP'},inplace = True)
geometry.rename(columns = {'GEOID':'FIPS GEOID'},inplace = True)

In [4]:
geometry.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,085,504321,06085504321,5043.21,Census Tract 5043.21,G5020,S,1450237,0,+37.3931319,-121.8651427,"POLYGON ((-121.87556 37.39924, -121.87535 37.3..."
1,06,085,504410,06085504410,5044.10,Census Tract 5044.10,G5020,S,1102136,0,+37.4093719,-121.8788884,"POLYGON ((-121.88886 37.40758, -121.88576 37.4..."


In [5]:
# loading in tract data
data = pd.read_csv(path + 'cleaned_tract_data.csv')

In [6]:
# defining functions for cleaning string
# please refer to FIPS makeup
def FIPS_cleaner(string):
    if len(string) == 10:
        fix = '0' + string
    elif len(string) == 11:
        fix = string
    else: return np.nan
    
    return fix

def fix_St(string):
    if (len(string) == 1):
        fix = '0' + string
    elif (len(string) == 2):
        fix = string
    else: return np.nan
    
    return fix

def fix_Cnty(string):
    if (len(string) == 1):
        fix = '00' + string
    elif (len(string) == 2):
        fix = '0' + string
    elif (len(string) == 3):
        fix = string
    else: return np.nan
    
    return fix

In [7]:
# converting to strings for FIPS merge consistency
data['FIPS'] = data['FIPS'].apply(lambda x: str(x)).apply(FIPS_cleaner)
data['state_fips'] = data['state_fips'].apply(lambda x: str(x)).apply(fix_St)
data['county'] = data['county'].apply(lambda x: str(x)).apply(fix_Cnty)
# fixing names
data.rename(columns = {'state_fips': 'STATEFP','county': 'COUNTYFP'}, inplace = True)

In [8]:
# converting population to numeric
data['total_pop'] = data["total_pop"].apply(lambda x: int(x))

In [9]:
# merging to append geometry
merge = geometry.merge(data, left_on = ['FIPS GEOID','STATEFP','COUNTYFP'], right_on = ['FIPS','STATEFP','COUNTYFP'], how="left")

In [10]:
# returning FIPS to numeric
merge['FIPS GEOID'] = merge['FIPS GEOID'].apply(lambda x: int(x))

In [11]:
# standardizing crs
merge.to_crs(pyproj.CRS.from_epsg(4326), inplace = True)

In [12]:
merge.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,FIPS,state,tract,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino
0,06,085,504321,6085504321,5043.21,Census Tract 5043.21,G5020,S,1450237,0,+37.3931319,-121.8651427,"POLYGON ((-121.87556 37.39924, -121.87535 37.3...",06085504321,ca,504321,14000US06085504321,Census Tract 5043.21,0.559940,5511,1779,1131,212,1316,60,146941.0,544,77,0,3559,0,7,269,1055,962,1675,0.635750,0.038469,0.045593,0.574328,0.098712,0.013972,0.000000,0.645799,0.0,0.00127,0.048811,0.191435
1,06,085,504410,6085504410,5044.10,Census Tract 5044.10,G5020,S,1102136,0,+37.4093719,-121.8788884,"POLYGON ((-121.88886 37.40758, -121.88576 37.4...",06085504410,ca,504410,14000US06085504410,Census Tract 5044.10,0.425537,4286,1244,688,189,934,102,128523.0,556,165,77,2476,0,0,94,918,589,1214,0.553055,0.044097,0.109208,0.485173,0.129725,0.038497,0.017965,0.577695,0.0,0.00000,0.021932,0.214186


- - -
## Checkpoint: 2. tract_w_geo.geojson

In [15]:
merge.to_file('tract_w_geo.geojson',driver = 'GeoJSON', crs = merge.crs)

In [16]:
merge = gpd.read_file('./Files/tract_w_geo.geojson')

In [17]:
merge.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,FIPS,state,tract,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino,geometry
0,06,085,504321,6085504321,5043.21,Census Tract 5043.21,G5020,S,1450237,0,+37.3931319,-121.8651427,06085504321,ca,504321,14000US06085504321,Census Tract 5043.21,0.559940,5511,1779,1131,212,1316,60,146941.0,544,77,0,3559,0,7,269,1055,962,1675,0.635750,0.038469,0.045593,0.574328,0.098712,0.013972,0.000000,0.645799,0.0,0.00127,0.048811,0.191435,"POLYGON ((-121.87556 37.39924, -121.87535 37.3..."
1,06,085,504410,6085504410,5044.10,Census Tract 5044.10,G5020,S,1102136,0,+37.4093719,-121.8788884,06085504410,ca,504410,14000US06085504410,Census Tract 5044.10,0.425537,4286,1244,688,189,934,102,128523.0,556,165,77,2476,0,0,94,918,589,1214,0.553055,0.044097,0.109208,0.485173,0.129725,0.038497,0.017965,0.577695,0.0,0.00000,0.021932,0.214186,"POLYGON ((-121.88886 37.40758, -121.88576 37.4..."


- - -

# 3. tracts_w_geo+.geojson
tract_w_geo.geojson, with additions:
1. column for county name

needed files:
1. tract_w_geo.geojson
2. county_w_geo.geojson

In [18]:
# reading in counties
counties = gpd.read_file('./Files/county_w_geo.geojson')

In [20]:
# merging to append county name to tract data, based on county FIPS code
tracts = merge.merge(counties[['COUNTYFP','county_name']], on = 'COUNTYFP', how = 'left')

In [21]:
tracts.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,FIPS,state,tract,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino,geometry,county_name
0,06,085,504321,6085504321,5043.21,Census Tract 5043.21,G5020,S,1450237,0,+37.3931319,-121.8651427,06085504321,ca,504321,14000US06085504321,Census Tract 5043.21,0.559940,5511,1779,1131,212,1316,60,146941.0,544,77,0,3559,0,7,269,1055,962,1675,0.635750,0.038469,0.045593,0.574328,0.098712,0.013972,0.000000,0.645799,0.0,0.00127,0.048811,0.191435,"POLYGON ((-121.87556 37.39924, -121.87535 37.3...",Santa Clara
1,06,085,504410,6085504410,5044.10,Census Tract 5044.10,G5020,S,1102136,0,+37.4093719,-121.8788884,06085504410,ca,504410,14000US06085504410,Census Tract 5044.10,0.425537,4286,1244,688,189,934,102,128523.0,556,165,77,2476,0,0,94,918,589,1214,0.553055,0.044097,0.109208,0.485173,0.129725,0.038497,0.017965,0.577695,0.0,0.00000,0.021932,0.214186,"POLYGON ((-121.88886 37.40758, -121.88576 37.4...",Santa Clara


In [22]:
tracts.to_file('tracts_w_geo+.geojson',driver = 'GeoJSON', crs = tracts.crs)

In [ ]:
test = gpd.read_file('./Files/tracts_w_geo+.geojson')
test

- - -
- - -

# 4. demographic_merge.geojson
needed files
1. tract_w_geo+.geojson
2. county_w_geo.geojson

In [32]:
# read in county data
counties = gpd.read_file('./Files/county_w_geo.geojson')

In [33]:
# read in tract data
tracts = gpd.read_file('./Files/tracts_w_geo+.geojson')

In [34]:
# concatenating dataframe
merge = pd.concat([tracts,counties])

In [35]:
merge.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,FIPS,state,tract,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino,county_name,geometry,COUNTYNS,StateCountyFIPS,LSAD,CLASSFP,CSAFP,CBSAFP,METDIVFP,geometry_type
0,06,085,504321,6.085504e+09,5043.21,Census Tract 5043.21,G5020,S,1450237,0,+37.3931319,-121.8651427,06085504321,ca,504321.0,14000US06085504321,Census Tract 5043.21,0.559940,5511,1779,1131,212,1316,60,146941.0,544,77,0,3559,0,7,269,1055,962,1675,0.635750,0.038469,0.045593,0.574328,0.098712,0.013972,0.000000,0.645799,0.0,0.00127,0.048811,0.191435,Santa Clara,"POLYGON ((-121.87556 37.39924, -121.87535 37.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06,085,504410,6.085504e+09,5044.10,Census Tract 5044.10,G5020,S,1102136,0,+37.4093719,-121.8788884,06085504410,ca,504410.0,14000US06085504410,Census Tract 5044.10,0.425537,4286,1244,688,189,934,102,128523.0,556,165,77,2476,0,0,94,918,589,1214,0.553055,0.044097,0.109208,0.485173,0.129725,0.038497,0.017965,0.577695,0.0,0.00000,0.021932,0.214186,Santa Clara,"POLYGON ((-121.88886 37.40758, -121.88576 37.4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
merge.to_file('demographic_merge.geojson',driver = 'GeoJSON', crs = merge.crs)

In [36]:
# test = gpd.read_file('./Files/demographic_merge.geojson')
# test

- - -
- - -

# 5-8. Pesticide_5.csv, pesticide_5_meridian.csv,
# Pesticide_5_w_tracts.csv,Pesticide_5_w_tracts+.csv
files needed:
1. pur2018_withtypes.csv
2. pur2019_withtypes.csv
3. pur2020_withtypes.csv
4. pur2021_withtypes.csv

In [11]:
# read in first two years of pesticide data, this already has been fuzzy matched for pesticide type
pd2018 = pd.read_csv("../../data/basefile/pur2018_withtypes.csv",low_memory = False,index_col=0)
pd2019 = pd.read_csv("../../data/basefile/pur2019_withtypes.csv",low_memory = False,index_col=0)

In [12]:
# read in next two years
pd2020 = pd.read_csv("../../data/basefile/pur2020_withtypes.csv",low_memory = False,index_col=0)
pd2021 = pd.read_csv("../../data/basefile/pur2021_withtypes.csv",low_memory = False,index_col=0)

In [13]:
# concatenating dataframes
Pesticide_5 = pd.concat([pd2018,pd2019,pd2020,pd2021])

In [14]:
Pesticide_5.head(2)

,ADJUVANT,YEAR,DATE,COUNTY_NAME,COMTRS,SITE_NAME,PRODUCT_NAME,POUNDS_PRODUCT_APPLIED,CHEMICAL_NAME,POUNDS_CHEMICAL_APPLIED,AMOUNT_TREATED,UNIT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Township,Range,Section,pur2020,chem_type,ratio,air,chem_code,chemname,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev
0,NO,2018,09-FEB-18,FRESNO,10M11S13E15,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),"2,4-DB 200 HERBICIDE",486.7475,"4-(2,4-DB), DIMETHYLAMINE SALT",127.527845,69.8,A,G,MDM,10,T11S,R13E,15.0,"4-(2,4-DB), DIMETHYLAMINE SALT",D DIMETHYLAMINE SALT,97.560976,True,806.0,"2,4-D, DIMETHYLAMINE SALT",False,False,False,False,False,False
1,NaN,2018,06-SEP-18,FRESNO,10M13S13E17,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),8-8-2,19.8856,NaN,NaN,75.0,A,A,MDM,10,T13S,R13E,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- - -
## Checkpoint: 5. Pesticide_5.csv

In [15]:
Pesticide_5.to_csv('Pesticide_5.csv')

In [3]:
Pesticide_5 = pd.read_csv('./Files/Pesticide_5.csv', low_memory = False, index_col = 0)

- - -

# 6. Pesticide_5_Meridian.csv
Pesticide_5, with additions:
1. added meridian
2. dropped any non-ag pesticides
3. focuses on subset of currently used columns

files needed
1. Pesticide_5.csv

In [4]:
# dropping incomplete comtrs: length should == 11 for agriculture
data = Pesticide_5.loc[Pesticide_5['COMTRS'].apply(len) == 11]

In [6]:
# creates TownshipRange column, look up PLSS township range system
data['TownshipRange'] = data['Township'] + " " + data['Range']

/tmp/ipykernel_2668372/750142773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TownshipRange'] = data['Township'] + " " + data['Range']


In [8]:
# taking character that is representative of meridian, we believe that our original files had incorrect meridian
data['Meridian'] = data['COMTRS'].str[2]
# relabeling to the full meridian name
data['Meridian'] = data['Meridian'].replace({'S':'SBM','M':'MDM','H':'HM'})

/tmp/ipykernel_2668372/512622698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Meridian'] = data['COMTRS'].str[2]
/tmp/ipykernel_2668372/512622698.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Meridian'] = data['Meridian'].replace({'S':'SBM','M':'MDM','H':'HM'})


In [10]:
# dropping unneeded columns
data.drop(columns = ['DATE','SITE_NAME','PRODUCT_NAME','CHEMICAL_NAME','POUNDS_CHEMICAL_APPLIED','UNIT_TREATED','Township','Range','pur2020','chem_type','ratio','chem_code','chemname'], inplace = True)

/tmp/ipykernel_2668372/476140911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns = ['DATE','SITE_NAME','PRODUCT_NAME','CHEMICAL_NAME','POUNDS_CHEMICAL_APPLIED','UNIT_TREATED','Township','Range','pur2020','chem_type','ratio','chem_code','chemname'], inplace = True)


In [11]:
# reseting the index
data.reset_index(inplace = True)
data.drop(columns = ['index'],inplace = True)

/tmp/ipykernel_2668372/3895165254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns = ['index'],inplace = True)


In [13]:
# code for converting county code to county fips
# THESE ARE NOT THE SAME THING
def Ccode_to_Cfp(num):
    Cfp = num * 2 - 1
    Cfp = str(Cfp)
    if (len(Cfp) == 1):
        fix = '00' + Cfp
    elif (len(Cfp) == 2):
        fix = '0' + Cfp
    elif (len(Cfp) == 3):
        fix = Cfp
    else: return np.nan

    return fix

# code to convert county fips to cleaned county fips string
def Cfp_to_CfpString(num):
    Cfp = str(num)
    if (len(Cfp) == 1):
        fix = '00' + Cfp
    elif (len(Cfp) == 2):
        fix = '0' + Cfp
    elif (len(Cfp) == 3):
        fix = Cfp
    else: return np.nan

    return fix

In [14]:
# creates county FIPS column
data['COUNTYFP'] = data.County_Code.apply(Ccode_to_Cfp)

/tmp/ipykernel_2668372/3664731223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['COUNTYFP'] = data.County_Code.apply(Ccode_to_Cfp)


In [15]:
# converting section to an integer
data['Section'] = data['Section'].apply(lambda x: int(x))

/tmp/ipykernel_2668372/3415772470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Section'] = data['Section'].apply(lambda x: int(x))


In [17]:
data

,ADJUVANT,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP
0,NO,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15,True,False,False,False,False,False,False,T11S R13E,019
1,NaN,2018,FRESNO,10M13S13E17,19.8856,75.0,A,MDM,10,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T13S R13E,019
2,NaN,2018,FRESNO,10M12S13E14,15.3373,57.9,A,MDM,10,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T12S R13E,019
3,NO,2018,FRESNO,10M15S16E18,74.9825,40.0,G,MDM,10,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T15S R16E,019
4,NO,2018,FRESNO,10M15S16E35,129.3698,69.0,G,MDM,10,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T15S R16E,019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4856494,NaN,2021,MONTEREY,27M14S02E24,4.1428,8.9,G,MDM,27,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T14S R02E,053
4856495,NO,2021,MONTEREY,27M20S08E05,82.6086,45.0,G,MDM,27,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T20S R08E,053
4856496,NaN,2021,MONTEREY,27M16S05E21,8.6431,9.5,G,MDM,27,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T16S R05E,053
4856497,NO,2021,MONTEREY,27M20S08E05,1.5837,45.0,G,MDM,27,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T20S R08E,053


- - -
## Checkpoint: 6. pesticide_5_meridian.csv

In [16]:
data.to_csv('pesticide_5_Meridian.csv')

In [18]:
data = pd.read_csv('./Files/pesticide_5_Meridian.csv',index_col = 0)

In [19]:
data.head(2)

,ADJUVANT,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP
0,NO,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15,True,False,False,False,False,False,False,T11S R13E,19
1,NaN,2018,FRESNO,10M13S13E17,19.8856,75.0,A,MDM,10,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T13S R13E,19


- - -

# 7. Pesticide_5_w_tracts.csv
Same as pesticide_5_meridian, with additions
1. added tract information relating to where pesticide is used

files needed
1. pesticide_5_Meridian.csv
2. mapper_sections_tracts_5_newtracts.csv

In [20]:
# read in mapper to get tracts for pesticide use
mapper = pd.read_csv('../../data/mapper_sections_tracts_5_newtracts.csv',index_col = 0)

In [21]:
# function used for cleaning townshiprange for creating COMTRS
# puts it into the same townshiprange format for COMTRS
def reduce(TownshipRange):
    fix = TownshipRange[1:]
    fix = fix.replace(' R','')
    return fix

# code to convert county fips to cleaned county fips string
def Cfp_to_CfpString(num):
    Cfp = str(num)
    if (len(Cfp) == 1):
        fix = '00' + Cfp
    elif (len(Cfp) == 2):
        fix = '0' + Cfp
    elif (len(Cfp) == 3):
        fix = Cfp
    else: return np.nan

    return fix

In [22]:
# turning county FIPS to string to use for COMTRS creation
mapper['COUNTYFP'] = mapper['COUNTYFP'].apply(Cfp_to_CfpString)

In [23]:
# creating COMTRS, this will be used to merge
mapper['COMTRS'] = (mapper['COUNTY_CODE'].apply(int)).apply(str) + mapper['Meridian'].str[0] + mapper['TownshipRange'].apply(reduce) + mapper['Section'].apply(lambda x: (str(x)).zfill(2))

In [24]:
# COMTRS captures all information from these columns, we drop them now
# geometry is not tract geometry, justifying drop
mapper.drop(columns = ['COUNTY_CODE','Meridian','Section', 'geometry','COUNTYFP','MTRS'], inplace = True)

In [25]:
# converting county code to county fips
data['COUNTYFP'] = data.County_Code.apply(Ccode_to_Cfp)

In [27]:
# return sections to integer
data['Section'] = data['Section'].apply(lambda x: int(x))

In [28]:
data.head(2)

,ADJUVANT,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP
0,NO,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15,True,False,False,False,False,False,False,T11S R13E,019
1,NaN,2018,FRESNO,10M13S13E17,19.8856,75.0,A,MDM,10,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T13S R13E,019


In [29]:
final_cut = pd.merge(data, mapper, left_on = ['COMTRS','TownshipRange'], right_on = ['COMTRS','TownshipRange'], how = 'inner')

In [30]:
# dropping columns with missing tract information
final_cut = final_cut.loc[final_cut['TRACTCE'].isna() == False]

In [36]:
final_cut.head(2)

,ADJUVANT,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP,TRACTCE
0,NO,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15,True,False,False,False,False,False,False,T11S R13E,19,8402
1,NO,2018,FRESNO,10M11S13E15,69.1503,69.8,G,MDM,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,19,8402


- - -
# Checkpoint: 7. Pesticide_5_w_tracts.csv

In [32]:
final_cut.to_csv('pesticide_5_w_tracts.csv')

In [34]:
final_cut = pd.read_csv('./Files/pesticide_5_w_tracts.csv', index_col = 0)

In [35]:
final_cut.head(2)

,ADJUVANT,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP,TRACTCE
0,NO,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15,True,False,False,False,False,False,False,T11S R13E,19,8402
1,NO,2018,FRESNO,10M11S13E15,69.1503,69.8,G,MDM,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,19,8402


- - -
- - -

# 8. pesticide_5_w_tracts+.csv
similiar to pesticide_5_w_Meridian, with additions:
1. makes FIPS code for tract pesticide was used in
2. makes a column that consolidates meridian and townshiprange

files needed:
1. pesticide_5_Meridian.csv
2. mapper_sections_tracts_5_newtracts.csv

The plus version of this file has very few differences, once difference is that we retain more columns and perform the merge using more columns, but also require less preprocessing of the mapping file, we foregoe making the COMTRS which summarizes many other columns. We also add a column that consolidates meridian and townshiprange, for plotting later

In [2]:
# reading in pesticide data
data = pd.read_csv('./Files/pesticide_5_Meridian.csv',index_col = 0)

In [3]:
# dropping columns we will not use in this case
data.drop(columns = ['COMTRS','AERIAL_GROUND_INDICATOR','County_Code',],inplace = True)

In [4]:
# read in mapper to get tracts for pesticide use
mapper = pd.read_csv('../../data/mapper_sections_tracts_5_newtracts.csv',index_col = 0)

In [5]:
# dropping geometry, not tract geometry
mapper.drop(columns = ['geometry'], inplace = True)

In [7]:
# merging to gain tract information
final_cut = pd.merge(data, mapper, on = ['COUNTYFP','Meridian','TownshipRange','Section'], how = 'left')

In [8]:
# drop any columns missing tract information
final_cut = final_cut.loc[final_cut['TRACTCE'].isna() == False]

In [10]:
# convering county FIPS code to string
def Cfp_to_CfpString(num):
    Cfp = str(num)
    if (len(Cfp) == 1):
        fix = '00' + Cfp
    elif (len(Cfp) == 2):
        fix = '0' + Cfp
    elif (len(Cfp) == 3):
        fix = Cfp
    else: return np.nan

    return fix

# converts tract into FIPS format
def tract_fixer(num):
    num = str(num)
    if (len(num) >= 2) & (len(num) <=6):
        zero_times = 6 - len(num)
        fix = zero_times*'0' + num
        return fix
    else: return np.nan

In [11]:
# convert tractce to integer values
final_cut['TRACTCE'] = final_cut['TRACTCE'].astype(int)

# convert county fips code to string
final_cut['COUNTYFP'] = final_cut['COUNTYFP'].apply(Cfp_to_CfpString)

# convert tract to tract string in FIPS format
final_cut['TRACTFP'] = final_cut['TRACTCE'].apply(tract_fixer)

# creating FIPS for where pesticide was applied
final_cut['PU_FIPS'] = '06' + final_cut['COUNTYFP'] + final_cut['TRACTFP']

# creates identifier that uses both township and meridian, for plotting uniqueness
# note that tractce, tract number is not unique for the state level, it should be unique up to the meridian however
# COMTRS should be even more 1-1, or PU_FIPS
final_cut['MTownshipRange'] = final_cut.Meridian + final_cut.TownshipRange

# creates generic count column, used for groupby counting
final_cut['Count'] = 1

# returning FIPS to integer form
final_cut['PU_FIPS'] = final_cut['PU_FIPS'].apply(int)

In [19]:
# dropping column that are summarized in other columns
final_cut.drop(columns = ['Meridian','Section','TownshipRange','COUNTYFP','TRACTCE','MTRS','TRACTFP'],inplace = True)

In [14]:
# resetting index
final_cut.reset_index(inplace = True)
final_cut.drop(columns = 'index', inplace = True)

In [18]:
final_cut

,ADJUVANT,YEAR,COUNTY_NAME,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,Meridian,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP,TRACTCE,MTRS,COUNTY_CODE,TRACTFP,PU_FIPS,MTownshipRange,Count
0,NO,2018,FRESNO,486.7475,69.8,MDM,15,True,False,False,False,False,False,False,T11S R13E,019,8402,MDM-T11S-R13E-15,10.0,008402,6019008402,MDMT11S R13E,1
1,NaN,2018,FRESNO,19.8856,75.0,MDM,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T13S R13E,019,8402,MDM-T13S-R13E-17,10.0,008402,6019008402,MDMT13S R13E,1
2,NaN,2018,FRESNO,15.3373,57.9,MDM,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T12S R13E,019,8405,MDM-T12S-R13E-14,10.0,008405,6019008405,MDMT12S R13E,1
3,NO,2018,FRESNO,74.9825,40.0,MDM,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T15S R16E,019,8200,MDM-T15S-R16E-18,10.0,008200,6019008200,MDMT15S R16E,1
4,NO,2018,FRESNO,129.3698,69.0,MDM,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T15S R16E,019,8200,MDM-T15S-R16E-35,10.0,008200,6019008200,MDMT15S R16E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4842576,NaN,2021,MONTEREY,4.1428,8.9,MDM,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T14S R02E,053,10504,MDM-T14S-R02E-24,27.0,010504,6053010504,MDMT14S R02E,1
4842577,NO,2021,MONTEREY,82.6086,45.0,MDM,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T20S R08E,053,11305,MDM-T20S-R08E-5,27.0,011305,6053011305,MDMT20S R08E,1
4842578,NaN,2021,MONTEREY,8.6431,9.5,MDM,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T16S R05E,053,14800,MDM-T16S-R05E-21,27.0,014800,6053014800,MDMT16S R05E,1
4842579,NO,2021,MONTEREY,1.5837,45.0,MDM,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T20S R08E,053,11305,MDM-T20S-R08E-5,27.0,011305,6053011305,MDMT20S R08E,1


In [20]:
final_cut.to_csv('pesticide_5_w_tracts+.csv')

In [21]:
test = pd.read_csv('./Files/pesticide_5_w_tracts+.csv')

In [22]:
test

,Unnamed: 0,ADJUVANT,YEAR,COUNTY_NAME,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,COUNTY_CODE,PU_FIPS,MTownshipRange,Count
0,0,NO,2018,FRESNO,486.7475,69.8,True,False,False,False,False,False,False,10.0,6019008402,MDMT11S R13E,1
1,1,NaN,2018,FRESNO,19.8856,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6019008402,MDMT13S R13E,1
2,2,NaN,2018,FRESNO,15.3373,57.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6019008405,MDMT12S R13E,1
3,3,NO,2018,FRESNO,74.9825,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6019008200,MDMT15S R16E,1
4,4,NO,2018,FRESNO,129.3698,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6019008200,MDMT15S R16E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4842576,4842576,NaN,2021,MONTEREY,4.1428,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,6053010504,MDMT14S R02E,1
4842577,4842577,NO,2021,MONTEREY,82.6086,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,6053011305,MDMT20S R08E,1
4842578,4842578,NaN,2021,MONTEREY,8.6431,9.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,6053014800,MDMT16S R05E,1
4842579,4842579,NO,2021,MONTEREY,1.5837,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,6053011305,MDMT20S R08E,1


- - -
- - -


# 9. places_w_MTownship.geojson
same as Public_Land_Survey_System_(PLSS)%3A_Township_and_Range.geojson, with additions:
1. added columns MTownshipRange, for 1-1 plotting
files needed
1. Public_Land_Survey_System_(PLSS)%3A_Township_and_Range.geojson

# NOTE:
The code commented out below is a means to open the file as a geojson object, useful for choropleth plotting with featureidkey parameter of choropleth maps

In [ ]:
# g = open('./places_w_MTownship.geojson')
# places2 = json.load(g)

In [ ]:
# reading in geometry information for township ranges
places = gpd.read_file('../../data/Public_Land_Survey_System_(PLSS)%3A_Township_and_Range.geojson')

In [ ]:
# this appends the township range with meridian information
places['MTownshipRange'] = places['Meridian'] + places['TownshipRange']

In [ ]:
places.to_file('places_w_MTownship.geojson', driver = 'GeoJSON', crs = places.crs)